In [1]:
import arcpy
from arcpy import env
import pandas as pd
import numpy as np

In [2]:
env_path = r"D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_ky.gdb"
env.workspace = env_path
raster_list = arcpy.ListRasters()
print(raster_list)
print(len(raster_list))

['PRE', 'SRA', 'TMP', 'VAP', 'WIND', 'BIO', 'Contrast', 'Correlation', 'Dissimilarity', 'Entropy', 'Homogeneity', 'Mean', 'ndvi', 'PCA_0', 'PCA_1', 'SecondMoment', 'Variance', 'LON', 'LAT', 'Aspect', 'ChannelNetworkBaseLevel', 'ChannelNetworkDistance', 'ClosedDepressions', 'ConvergenceIndex', 'LSFactor', 'PlanCurvature', 'ProfileCurvature', 'RelativeSlopePosition', 'Slope', 'TopographicPositionIndex', 'TopographicWetnessIndex', 'TotalCatchmentArea', 'ValleyDepth', 'DEM', 'AnalyticalHillshading', 'DL', 'ASP_a', 'TWI3_a', 'TWI5_a', 'TPI3_a', 'TPI5_a', 'DZ']
42


In [3]:
# 检查现有的raster空间范围和像元是否一致

def check_raster_info(raster_path):
    desc = arcpy.Describe(raster_path)
    origin_x = desc.extent.XMin
    origin_y = desc.extent.YMax
    pixel_width = desc.meanCellWidth
    pixel_height = desc.meanCellHeight
    rows = desc.height
    cols = desc.width
    print(origin_x,origin_y,rows,cols,pixel_width,pixel_height)
    return((origin_x,origin_y,rows,cols,pixel_width,pixel_height))

In [4]:
check_list = set()
for one_raster in raster_list:
    check_raster_info(one_raster)
    check_list.add(check_raster_info(one_raster))

370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 5.0 5.0
370777.4999999997 3030847.5 13978 11971 

In [5]:
check_list

{(370777.4999999997, 3030847.5, 13978, 11971, 5.0, 5.0)}

In [8]:
# 选取标准数据生成lon、lat图层
stander_raster = "dem"
stander_desc = arcpy.Describe(stander_raster)
stander_origin_x = stander_desc.extent.XMin
stander_origin_y = stander_desc.extent.YMax
stander_pixel_width = stander_desc.meanCellWidth
stander_pixel_height = stander_desc.meanCellHeight
stander_array = arcpy.RasterToNumPyArray("dem")

In [9]:
# 经度
array_x = np.zeros(stander_array.shape,dtype=np.float32)
array_x[:, 0] = stander_origin_x+stander_pixel_width/2
for i in range(1, stander_array.shape[1]):
    array_x[:, i] = array_x[:, i-1] + stander_pixel_width
print(array_x.shape,array_x[0,0],array_x[-1,-1])

(13978, 11971) 370780.0 430630.0


In [10]:
# 纬度
array_y = np.zeros(stander_array.shape,dtype=np.float32)
array_y[0] = stander_origin_y-stander_pixel_height/2
for i in range(1, stander_array.shape[0]):
    array_y[i] = array_y[i-1] - stander_pixel_height
print(array_y.shape,array_y[0][0],array_y[-1][-1])

(13978, 11971) 3030845.0 2960960.0


In [12]:
# 确定范围
env.extent = "dem"

In [13]:
original_raster = arcpy.Raster("dem")

In [14]:
# 将生成的x,y数组保存为栅格
out_raster_x = arcpy.NumPyArrayToRaster(
    array_x,
    lower_left_corner=original_raster.extent.lowerLeft,
    x_cell_size=original_raster.meanCellWidth,
    y_cell_size=original_raster.meanCellHeight,
    )
out_raster_y = arcpy.NumPyArrayToRaster(
    array_y,
    lower_left_corner=original_raster.extent.lowerLeft,
    x_cell_size=original_raster.meanCellWidth,
    y_cell_size=original_raster.meanCellHeight,
    )

In [15]:
out_raster_x.save("LON")
out_raster_y.save("LAT")

In [16]:
spatial_ref = arcpy.Describe("DEM").spatialReference

In [17]:
arcpy.DefineProjection_management("LON", spatial_ref)
arcpy.DefineProjection_management("LAT", spatial_ref)

<Result 'D:\\ArcGISProjects\\workspace\\shbyq\\feature_raster_file\\features_data_ky.gdb\\LAT'>